## Análise KMEP para semana

In [1]:
import numpy as np
import pandas as pd
import pyodbc
from datetime import date
from datetime import datetime
import re

In [2]:
# defini servidor, nome de usuario e senha
server = '10.200.92.45' 
database = 'ALL_STAGE' 
username = 'appBIRumoRead' 
password = 'R******' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# tabela de mesas da operação norte
df_paradas = pd.read_sql_query(
'''SELECT
    p.MOTIVO_PARADA,
    p.PREFIXO_TREM,
    p.OS_TREM,
    p.ID_MAQUINISTA,
    p.KM,
    p.DATA_PARADA,
    p.SB As SB,
    m.Mesa,
    DATEPART( wk, p.DATA_PARADA) AS NUM_SEMANA
FROM px.ParadaTrem p, atw.MESA_DIVISION m (NOLOCK) 
WHERE m.Mesa in ('SANTISTA','RONDONOPOLIS', 'BOA VISTA', 'ARARAQUARA', 'PORTOFER') AND m.SBATW=p.SB 
      AND YEAR(p.DATA_PARADA)=2021 AND MONTH(p.DATA_PARADA)=5 AND p.OS_TREM<>0 AND LEFT(p.PREFIXO_TREM,1) NOT IN ('M','U')
''', cnxn)
df_paradas.drop_duplicates(subset=['OS_TREM','KM','ID_MAQUINISTA'],inplace=True)

In [15]:
df_paradas.head(10)

,MOTIVO_PARADA,PREFIXO_TREM,OS_TREM,ID_MAQUINISTA,KM,DATA_PARADA,SB,Mesa,NUM_SEMANA,Trecho,Sentido,Mes
0,OPERAÇÃO,L65,1778330.0,257483.0,75.630,2021-05-22 12:47:33,K79ZEZ1,PORTOFER,21,TM 1: ZEM-ZPT,Importação,5
1,OPERAÇÃO,O57,1775215.0,263050.0,75.291,2021-05-07 16:11:30,K79ZEZ1,PORTOFER,19,TM 1: ZEM-ZPT,Importação,5
2,OPERAÇÃO,L63,1777920.0,289081.0,75.630,2021-05-20 22:32:49,K79ZEZ1,PORTOFER,21,TM 1: ZEM-ZPT,Importação,5
3,OPERAÇÃO,L97,1777267.0,13285.0,75.291,2021-05-17 15:29:48,K79ZEZ1,PORTOFER,21,TM 1: ZEM-ZPT,Importação,5
4,OPERAÇÃO,X03,1778924.0,261452.0,76.043,2021-05-25 14:35:44,K79ZEZ1,PORTOFER,22,TM 1: ZEM-ZPT,Importação,5
5,OPERAÇÃO,L97,1778180.0,257297.0,75.743,2021-05-22 05:00:55,K79ZEZ1,PORTOFER,21,TM 1: ZEM-ZPT,Importação,5
6,OPERAÇÃO,X15,1779130.0,256481.0,75.630,2021-05-26 08:17:07,K79ZEZ1,PORTOFER,22,TM 1: ZEM-ZPT,Importação,5
7,CCO,L97,1779741.0,259256.0,75.630,2021-05-29 05:43:24,K79ZEZ1,PORTOFER,22,TM 1: ZEM-ZPT,Importação,5
8,OPERAÇÃO,L97,1776073.0,261919.0,75.630,2021-05-12 01:22:57,K79ZEZ1,PORTOFER,20,TM 1: ZEM-ZPT,Importação,5
9,OPERAÇÃO,C63,1776074.0,256481.0,75.630,2021-05-12 05:19:05,K79ZEZ1,PORTOFER,20,TM 1: ZEM-ZPT,Importação,5


In [7]:
# defini servidor, nome de usuario e senha
server = '10.200.92.45' 
database = 'ALL_STAGE' 
username = 'appBIRumoRead' 
password = 'R**********' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

params = ('2021-05-01','2021-05-31')

# tabela de macros
df_TC = pd.read_sql_query(
'''SELECT  
    SECAO_BLOQUEIO as SB,
    TRECHO as Trecho,
    DTINICIO as DataInicio,
    CONDUCAO as Conducao,
    PREFIXO as Prefixo
FROM  px.TRANSIT_TIME
WHERE DTINICIO BETWEEN ? AND ?
''',cnxn,params=params)

quebra_tm = ['TM 2: ZYU-ZEM', 'TM 8: ZSF-ZZM', 'TM 9: TIN-ZSF', 'TM 0: ZPT-PSN/PCZ', 'TM 5: ZSD-ZAR',
       'TM 4: ZAR-ZRX', 'TM 7: ZZM-ZRU',  'TM 3: ZRX-ZYU','TM 12: TIQ-TAG',
       'TM 1: ZEM-ZPT', 'TM 11: TAG-TCS', 'TM 10: TCS-TIN','TM 6: ZRU-ZSD',  'TM 13: TRO-TIQ']
df_sbs_trecho = df_TC[df_TC.Trecho.isin(quebra_tm)][['SB','Trecho']].drop_duplicates(subset=['SB'])

In [8]:
# carrega bases de mapas
df_mapa_2 = pd.read_excel(r'\\ALLSDCLSFSR02.all-logistica.net\Engenharia$\05.Operacao Norte\Outros\Bases Importantes/Mapa.xlsx',sheet_name='AREAOPER - TL')
# cria colunas de trecho, sentido e mes
df_paradas = df_paradas.merge(df_sbs_trecho,on='SB')
df_paradas['Sentido'] = ''
df_paradas.loc[df_paradas['PREFIXO_TREM'].str.contains('^[A-Z]{1}[0-9]{1}[02468]'),['Sentido']] = 'Exportação'
df_paradas.loc[df_paradas['PREFIXO_TREM'].str.contains('^[A-Z]{1}[0-9]{1}[13579]'),['Sentido']] = 'Importação'
df_paradas['Mes'] = df_paradas.DATA_PARADA.apply(lambda x:x.month)

# carrega base de distancias
arquivos_excel = ['S:/PCP Ferro/5.Controle/8. Indicadores Op Norte/24. Painel Diesel/Cálculo/05. Painel Diesel MAI.xlsm']
df_TKB = pd.DataFrame()
for f in arquivos_excel:
    data = pd.read_excel(f,sheet_name='baseTKB',header=1,parse_dates=True,usecols=['DATA','DISTANCIA','TREM','AREAOPER'])
    df_TKB = df_TKB.append(data)
df_TKB.reset_index(drop=True,inplace=True)  
# deleta prefixos de congeneres
df_TKB.drop(df_TKB[df_TKB.TREM.str.contains('^[UM]')].index,inplace=True)
# cria colunas de trecho, sentido e mes

df_TKB = df_TKB.merge(df_mapa_2,left_on='AREAOPER',right_on='AREAOPER')
df_TKB['Sentido'] = ''
df_TKB.loc[df_TKB['TREM'].str.contains('^[A-Z]{1}[0-9]{1}[02468]'),['Sentido']] = 'Exportação'
df_TKB.loc[df_TKB['TREM'].str.contains('^[A-Z]{1}[0-9]{1}[13579]'),['Sentido']] = 'Importação'
df_TKB['Mes'] = df_TKB.DATA.apply(lambda x:x.month)

In [9]:
# agrupa para comparação importação
df_summ_paradas_imp = df_paradas[['Mes','Trecho','Sentido',
                                  'MOTIVO_PARADA']][df_paradas['Sentido']=='Importação'].groupby(by=['Mes','Trecho']).count().reset_index().rename(columns={'MOTIVO_PARADA':'Quantidade'})
df_summ_paradas_exp = df_paradas[['Mes','Trecho','Sentido',
                                  'MOTIVO_PARADA']][df_paradas['Sentido']=='Exportação'].groupby(by=['Mes','Trecho']).count().reset_index().rename(columns={'MOTIVO_PARADA':'Quantidade'})

# agrupa para comparação exportação
df_summ_tkb_imp = df_TKB[(df_TKB['Sentido']=='Importação') & (df_TKB['Trecho TM']!='Central')].groupby(by=['Mes','Trecho TM']).sum().reset_index().rename(columns={'DISTANCIA':'Distancia'})
df_summ_tkb_exp = df_TKB[(df_TKB['Sentido']=='Exportação') & (df_TKB['Trecho TM']!='Central')].groupby(by=['Mes','Trecho TM']).sum().reset_index().rename(columns={'DISTANCIA':'Distancia'})

# une dados para exportção
df_KMEP_imp = pd.concat([df_summ_paradas_imp.drop(columns=['Sentido']),df_summ_tkb_imp.drop(columns=['Mes','Trecho TM'])],axis=1)

# une dados para exportção
df_KMEP_exp = pd.concat([df_summ_paradas_exp.drop(columns=['Sentido']),df_summ_tkb_exp.drop(columns=['Mes','Trecho TM'])],axis=1)

In [10]:
# une dados para exportção
df_KMEP_imp = pd.concat([df_summ_paradas_imp.drop(columns=['Sentido']),df_summ_tkb_imp.drop(columns=['Mes','Trecho TM'])],axis=1)
df_KMEP_imp['KMEP'] = df_KMEP_imp['Distancia']/df_KMEP_imp['Quantidade']
df_KMEP_imp['Num TM'] = df_KMEP_imp['Trecho'].apply(lambda x:int(re.findall('[\d]+',x)[0]))

# une dados para exportção
df_KMEP_exp = pd.concat([df_summ_paradas_exp.drop(columns=['Sentido']),df_summ_tkb_exp.drop(columns=['Mes','Trecho TM'])],axis=1)
df_KMEP_exp['KMEP'] = df_KMEP_exp['Distancia']/df_KMEP_exp['Quantidade']
df_KMEP_exp['Num TM'] = df_KMEP_exp['Trecho'].apply(lambda x:int(re.findall('[\d]+',x)[0]))


In [11]:
df_KMEP_imp.sort_values(by='Num TM').drop(columns=['Num TM']).to_excel(r'C:\Users\cs322055\OneDrive - Rumo\Documentos\3. Area de transferência\KMEP\KMEP_imp.xlsx',index=False)

In [12]:
df_KMEP_exp.sort_values(by='Num TM').drop(columns=['Num TM']).to_excel(r'C:\Users\cs322055\OneDrive - Rumo\Documentos\3. Area de transferência\KMEP\KMEP_exp.xlsx',index=False)